In [1]:
import pyGPL
import numpy as np
import pandas as pd
import re

### First, we load in the example integrals in $ G(a_1, a_2, \dots, a_n, x) $ form. 
### The first two entries are loop 2 and 3. The last two entries are loop 4, respectively, without and with the basis functions, including $ \text{Li}_{2,2} $.


In [2]:
# Read the file content
with open("exampleIntegrals.txt", "r") as file:
    data = file.read()

# Extract expressions enclosed in `{}` as separate entries
blocks = re.findall(r"\{(.*?)\}", data, re.DOTALL)

# Create a DataFrame where each row corresponds to a `{}` block
df = pd.DataFrame(blocks, columns=["Integral"])

df['Integral'][0]

'G[0, 0, x], G[1, 0, x], G[-1, 0, x], G[4, 0, x], G[-4, 0, x], G[0, 1, x], G[1, 1, x], G[-1, 1, x], G[4, 1, x], G[-4, 1, x], G[0, -1, x], G[1, -1, x], G[-1, -1, x], G[4, -1, x], G[-4, -1, x], G[0, 4, x], G[1, 4, x], G[-1, 4, x], G[4, 4, x], G[-4, 4, x], G[0, -4, x], G[1, -4, x], G[-1, -4, x], G[4, -4, x], G[-4, -4, x]'

### Now we parse each entry so we have the GPLs in a suitable form to pass to fastGPL. $ G(a_1, a_2, \dots, a_n, x) \to [a_1, a_2, \dots, a_n]$

In [3]:
import re

def parse_integrals(integral_string):
    """
    Parses a string of integrals like 'G[a, b, c, x]' and extracts only the numerical lists.

    Args:
        integral_string (str): A string of comma-separated integrals in 'G[ ... ]' format.

    Returns:
        list: A list of lists containing only the numerical values from each G expression.
    """
    # Use regex to extract everything inside G[ ... ]
    matches = re.findall(r"G\[(.*?)\]", integral_string)
    
    # Process each match to extract only numeric values (ignore 'x')
    parsed_values = []
    for match in matches:
        values = match.split(",")  # Split by comma
        numeric_values = [int(v.strip()) for v in values if "x" not in v]  # Exclude 'x'
        parsed_values.append(numeric_values)

    return parsed_values

# Example usage
example_str = "G[0, 0, x], G[1, 0, x], G[-1, 0, x], G[4, 0, x], G[-4, 0, 1, x]"
parsed_list = parse_integrals(example_str)

print(parsed_list)  # Output: [[0, 0], [1, 0], [-1, 0], [4, 0], [-4, 0]]


[[0, 0], [1, 0], [-1, 0], [4, 0], [-4, 0, 1]]


In [4]:
# Apply the parse_integrals function to transform each row in the DataFrame
df["Parsed"] = df["Integral"].apply(parse_integrals)
df["Parsed"][0]

[[0, 0],
 [1, 0],
 [-1, 0],
 [4, 0],
 [-4, 0],
 [0, 1],
 [1, 1],
 [-1, 1],
 [4, 1],
 [-4, 1],
 [0, -1],
 [1, -1],
 [-1, -1],
 [4, -1],
 [-4, -1],
 [0, 4],
 [1, 4],
 [-1, 4],
 [4, 4],
 [-4, 4],
 [0, -4],
 [1, -4],
 [-1, -4],
 [4, -4],
 [-4, -4]]

### We can now evaluate the GPL for each $[a_1, a_2, \dots, a_n]$ and sum

In [5]:
#select arbitrary point to evaluate GPLs
point = 1.5

def evaluate_gpl(values):
    return pyGPL.GPL([complex(z) for z in values], point)

#test function
evaluate_gpl([1,0])

(2.0933482737718725+0j)

In [6]:
#seperate each integral

I1 = pd.DataFrame({"G": df["Parsed"][0]})
I2 = pd.DataFrame({"G": df["Parsed"][1]})
I3 = pd.DataFrame({"G": df["Parsed"][2]})
I4 = pd.DataFrame({"G": df["Parsed"][3]})

print('First few values of first integral:')
I1.head()

First few values of first integral:


,G
0,"[0, 0]"
1,"[1, 0]"
2,"[-1, 0]"
3,"[4, 0]"
4,"[-4, 0]"


In [7]:
I1["Eval"] = I1["G"].apply(evaluate_gpl)
I1.head()

,G,Eval
0,"[0, 0]",0.082201+0.000000j
1,"[1, 0]",2.093348+0.000000j
2,"[-1, 0]",-0.775857+0.000000j
3,"[4, 0]",0.227085+0.000000j
4,"[-4, 0]",-0.215581+0.000000j


In [8]:
print(f'First integral evaluated at x = {point}: {I1['Eval'].sum()}')

First integral evaluated at x = 1.5: (-4.821009669680437+1.4987242998155623j)


In [9]:
I2["Eval"] = I2["G"].apply(evaluate_gpl)
I2.head()

,G,Eval
0,"[0, 0, 0]",0.011110+0.000000j
1,"[1, 0, 0]",-1.155120+0.000000j
2,"[-1, 0, 0]",0.907935+0.000000j
3,"[4, 0, 0]",-0.264207+0.000000j
4,"[-4, 0, 0]",0.245527+0.000000j


In [10]:
I3["Eval"] = I3["G"].apply(evaluate_gpl)
I3.head()

,G,Eval
0,"[0, 0, 0, 0]",0.001126+0.000000j
1,"[1, 0, 0, 0]",1.086620+0.000000j
2,"[-1, 0, 0, 0]",-0.946379+0.000000j
3,"[4, 0, 0, 0]",0.253517+0.000000j
4,"[-4, 0, 0, 0]",-0.245983+0.000000j


In [11]:
I4["Eval"] = I4["G"].apply(evaluate_gpl)
I4.head()

,G,Eval
0,"[0, 0, 0, 0, 0]",0.000091+0.000000j
1,"[1, 0, 0, 0, 0]",-1.036598-0.000000j
2,"[-1, 0, 0, 0, 0]",0.972173+0.000000j
3,"[4, 0, 0, 0, 0]",-0.252071+0.000000j
4,"[-4, 0, 0, 0, 0]",0.248131+0.000000j


In [12]:
from IPython.display import display, Math

# Display the formatted output in LaTeX
display(Math(rf"""
\textbf{{Integrals evaluated at }} x = {point}:

\begin{{aligned}}
\text{{Loop 2:}} & \quad {I1["Eval"].sum():.4f} \\
\text{{Loop 3:}} & \quad {I2["Eval"].sum():.4f} \\
\text{{Loop 4:}} & \quad {I3["Eval"].sum():.4f} \\
\text{{Loop 4:}} & \quad {I4["Eval"].sum():.4f} \text{{  (with }} \text{{Li}}_{{2,2}} \text{{)}}
\end{{aligned}}
"""))


<IPython.core.display.Math object>